# 2. 이미지 다루기
* **2.1 색공간 변경**
* **2.2 형태 변경** : Resize, Rotate, Transform
* **2.3 블러** : Smoothing/Blur

## 2.1  색공간 변경
* `cv2.cvtColor()`, `cv2.inRange()` 등의 함수 이용
* BGR - Gray , BGR - HSV 등의 색공간 변경가능

### OpenCV에서 지원하는 변경가능 색공간 목록

In [ ]:
import cv2
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags)

### 색공간 변경 예제
* imread 로 읽어들인 기본 이미지는 BGR 형식의 색공간을 가진다. 화면에 표시하려면 RGB로 바꿀 필요가 있다.
* gray로 변경한 경우 단일색상을 표현할 컬러맵을 지정하여 보일 수 있다.<br>
 단순 회색색상이 아니라 사전정의된 팔레트를 사용하면 열적외선 카메라로 촬영된 이미지도 보기 쉽게 표시 할 수 있을 듯 함
* 다양한 분석을 위해 사용할 수 있는 [HSV 색공간(색상/명도/채도 기반)](https://ko.wikipedia.org/wiki/HSV_%EC%83%89_%EA%B3%B5%EA%B0%84)도 알아두자

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('images/phone.jpg')

# 가로16 * 세로12 패널 생성 
fig = plt.figure(figsize=(16,12))

# 원본 BGR이미지
ax1 = fig.add_subplot(2, 2, 1)
ax1.axis('off')
ax1.set_title('Origin - BGR')
ax1.imshow(img)

# BGR to RGB
img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
ax2 = fig.add_subplot(2, 2, 2)
ax2.set_title('BGR to RGB')
ax2.axis('off')
ax2.imshow(img2)

# BGR to Gray
# 초록색상으로 이미지가 나오는 경우, matplotlib의 컬러맵을 회색색상으로 지정
img3 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ax3 = fig.add_subplot(2, 2, 3)
ax3.set_title('BGR to Gray')
ax3.axis('off')
ax3.imshow(img3, cmap=plt.get_cmap('gray'))

# BGR to HSV
img4 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
ax4 = fig.add_subplot(2, 2, 4)
ax4.set_title('BGR to HSV')
ax4.axis('off')
ax4.imshow(img4)

plt.show()

### 물체 추적
* 파란색 물체를 추출하는 샘플코드
* 영상의 색공간을 HSV로 변환 후 파란색에 해당하는 Hue 110~130 영역을 마스킹으로 뽑아내어 원본영상에 덧씌움

In [ ]:
%pylab inline 
import cv2
from IPython.display import clear_output
from IPython.display import display
import numpy as np

try:
    cap = cv2.VideoCapture(1)
    while(1):
        # Take each frame
        _, frame = cap.read() 

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # define range of blue color in HSV
        lower_blue = np.array([110,50,90])
        upper_blue = np.array([130,255,255])

        # Threshold the HSV image to get only blue colors
        mask = cv2.inRange(hsv, lower_blue, upper_blue) 

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(rgb,rgb, mask= mask)
        
        # Display the frame
        figure(figsize=(12, 12))
        imshow(res)
        axis('off')
        show()
        # Display the frame until new frame is available
        clear_output(wait=True)
except KeyboardInterrupt:
    # Release the Video Device
    cap.release()


## 2.2 이미지 형태 변경

### 크기 변경
* `cv2.resize()` 함수를 이용해서 크기 변경 가능
* 보간방법(interpolation) 선택 가능 : `cv2.INTER_AREA`(축소), `cv2.INTER_CUBIC`(줌/느림), `cv2.INTER_LINEAR`(줌/기본값)

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt

bgr = cv2.imread('images/phone.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

## 원본이미지
print("origin size : {}".format(img.shape[:2]))

## 확대방법 1
res = cv2.resize(img,None,fx=2, fy=2, interpolation = cv2.INTER_CUBIC)
print("res-1 size : {}".format(res.shape[:2]))

## 확대방법 2
height, width = img.shape[:2]
res2 = cv2.resize(img,(2*width, 2*height), interpolation = cv2.INTER_CUBIC)
print("res-2 size : {}".format(res2.shape[:2]))


### 평행이동
* `cv2.warpAffine()` 함수 이용
* 원본이미지, 이미지변환행렬, 결과이미지 크기를 파라메터로 받음
$$
M = \begin{bmatrix}
1 & 0 & t_x\\ 
0 & 1 & t_y
\end{bmatrix}
$$

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

bgr = cv2.imread('images/phone.jpg')
img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 변환할 크기와 변환행렬
rows,cols,_ = bgr.shape
M = np.float32([[1,0,100],[0,1,50]])

# 변환
dst = cv2.warpAffine(img,M,(cols,rows))

# 화면출력
plt.figure(figsize=(8,6))
plt.axis('off')
plt.imshow(dst)
plt.show()

### 회전
* 변환행렬을 구할 때 `cv2.getRotationMatrix2D` 함수 이용
* 화전도 변환작업은 `cv2.warpAffine()` 함수 이용

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

bgr = cv2.imread('images/phone.jpg')
img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 변환할 크기와 변환행렬(90도 회전)
rows,cols,_ = bgr.shape
M = cv2.getRotationMatrix2D((cols/2,rows/2),90,1)

# 변환
dst = cv2.warpAffine(img,M,(cols,rows))

# 화면출력
plt.figure(figsize=(8,6))
plt.axis('off')
plt.imshow(dst)
plt.show()

### 아핀변환 (Affine Transformation)
* 두 공간 사이의 선형변환. 원본기준점에서 대상기준점으로 대응되는 확대/축소/기울기/회전등이 복합적으로 수행된 선형변환
* `cv2.getAffineTransform`함수를 이용해서 변환행렬 생성 가능

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

bgr = cv2.imread('images/phone.jpg')
img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 시작기준점 표시
img = cv2.circle(img,(50-5,50-5), 10, (0,255,0), -1)
img = cv2.circle(img,(200-5,50-5), 10, (255,255,0), -1)
img = cv2.circle(img,(100-5,250-5), 10, (255,0,0), -1)

# 변환할 크기와 변환행렬
rows,cols,_ = img.shape
pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,200],[200,50],[100,250]])
M = cv2.getAffineTransform(pts1,pts2)

# 변환
dst = cv2.warpAffine(img,M,(cols,rows))

# 화면출력
fig = plt.figure(figsize=(16,6))
ax1 = fig.add_subplot(1, 2, 1)
ax1.axis('off'), ax1.set_title('Origin'), ax1.imshow(img)
ax2 = fig.add_subplot(1, 2, 2)
ax2.axis('off'), ax2.set_title('AffineTransform'), ax2.imshow(dst)
plt.show()

### 원근변환 (Perspective Transformation)
* 4개의 점을 기준으로 내부의 이미지를 맞춰 변환
* 4개의 가운데 3개의 점이 동시에 한직선안에 존재하면 안됨
* `cv2.getPerspectiveTransform`함수를 이용해 변환행렬 생성가능
* `cv2.warpPerspective`함수 이용해 변환 수행

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

bgr = cv2.imread('images/sudoku.jpg')
img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 기준점 
org_pts = [[140,92],[370,100],[127,336],[385,338]];
pts1 = np.float32(org_pts)
pts2 = np.float32([[0,0],[300,0],[0,300],[300,300]])

# 시작기준점 표시
for p in org_pts:
    img = cv2.circle(img, tuple(p) , 5, (0,255,0), -1)

# 변환행렬
M = cv2.getPerspectiveTransform(pts1,pts2)

# 변환
dst = cv2.warpPerspective(img,M,(300,300))

# 화면출력
fig = plt.figure(figsize=(16,6))
ax1 = fig.add_subplot(1, 2, 1)
ax1.axis('off'), ax1.set_title('Origin'), ax1.imshow(img)
ax2 = fig.add_subplot(1, 2, 2)
ax2.axis('off'), ax2.set_title('PerspectiveTransform'), ax2.imshow(dst)
plt.show()

## 2.3 이미지 부드럽게 하기 (뭉개기)

### 2D Convolution (이미지 필터)
* low-pass filters (LPF) : 노이즈제거, 이미지 블러(blur)에 사용
* high-pass filters (HPF) : 경계(edge)검출에 사용
* `cv2.filter2D()`를 이용해서 5*5커널크기를 갖는 평균값 필터를 적용하면 이미지 흐리기를 할 수 있음
$$
M = \frac{1}{25}\begin{bmatrix}
1 & 1 & 1 & 1 & 1\\ 
1 & 1 & 1 & 1 & 1\\ 
1 & 1 & 1 & 1 & 1\\ 
1 & 1 & 1 & 1 & 1\\ 
1 & 1 & 1 & 1 & 1
\end{bmatrix}
$$

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

bgr = cv2.imread('images/phone.jpg')
img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 5*5 커널
kernel = np.ones((5,5),np.float32)/25

# 필터적용
dst = cv2.filter2D(img,-1,kernel)


# 화면출력
fig = plt.figure(figsize=(12,16))
ax1 = fig.add_subplot(2, 1, 1)
ax1.axis('off'), ax1.set_title('Origin'), ax1.imshow(img)
ax2 = fig.add_subplot(2, 1, 2)
ax2.axis('off'), ax2.set_title('Filtered'), ax2.imshow(dst)
plt.show()

### 이미지 블러(Blur)
* 평균값 : `cv2.blur()` 또는 `cv2.boxFilter()` 사용
* 가우시안 필터 : `cv2.GaussianBlur()` 사용. 필터수정을 위한 가우시안 커널을 구하려면 `cv2.getGaussianKernel()`이용 가능
* 메디안 필터 : 커널의 평균값기준 필터. `cv2.medianBlur()`사용가능. 노이즈 첨가될 수 있음  
* 양측 필터(Bilateral Filtering) : 엣지 주변을 부드럽게 함. `cv2.bilateralFilter()`

In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

# 원본
bgr = cv2.imread('images/night_road.jpg')
img = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

# 평균값 블러
avg_blur = cv2.blur(img,(10,10))

# 가우시안
gau_blur = cv2.GaussianBlur(img,(5,5),0)

# 메디안
median = cv2.medianBlur(img,5)

# Bilateral
bilateral = cv2.bilateralFilter(img,9,75,75)

# 화면출력
fig = plt.figure(figsize=(15,30))
ax1 = fig.add_subplot(3, 2, 1)
ax1.axis('off'), ax1.set_title('Origin'), ax1.imshow(img)
ax2 = fig.add_subplot(3, 2, 2)
ax2.axis('off'), ax2.set_title('Average'), ax2.imshow(avg_blur)
ax3 = fig.add_subplot(3, 2, 3)
ax3.axis('off'), ax3.set_title('Gaussian Filter'), ax3.imshow(gau_blur)
ax4 = fig.add_subplot(3, 2, 4)
ax4.axis('off'), ax4.set_title('Median Filter'), ax4.imshow(median)
ax5 = fig.add_subplot(3, 2, 5)
ax5.axis('off'), ax5.set_title('Bilateral Filter'), ax5.imshow(bilateral)
plt.show()